<a href="https://colab.research.google.com/github/ssgalitsky/PedalNerd/blob/main/PedalNerd_S1E1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

  

 | W$_Я$a${_P}$ **C** o$^{1_A}b$  |  <img width=4000px/>   |     $^{(C)\ 2020,\  Sergey\ Galitsky.}_{Brain,\ Music,\ Hatered}$ |
| :-------------------------------   |  :-----------------------------------:  | --------------------:  |


In [ ]:
#@markdown         ## **PedalNerd #s1e1**  
#@markdown ##   training WaveNet on Colab

#@markdown Parameters:

#@markdown - Use Google TPU option 
UseTPU = True  #@param {type: "boolean"}

#!rm -rf '/content/PedalNetRT'
#import os
#os.chdir('/content')
#!git clone https://github.com/ssgalitsky/PedalNetRT.git
!pip3 install torchvision
!pip3 install pytorch_lightning==0.9.0

if UseTPU:
  !curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
  !python3 pytorch-xla-env-setup.py --version nightly --apt-packages libomp5 libopenblas-dev
  !pip3 install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.6-cp36-cp36m-linux_x86_64.whl

import os
import torch
use_cuda=True
if use_cuda and torch.cuda.is_available():
  net.cuda()

if use_tpu:
  assert os.environ['COLAB_TPU_ADDR']

In [ ]:
#@title 3. prepare data
os.chdir('/content/PedalNetRT')
!python3 "prepare_data.py" "/content/PedalNetRT/data/s chain1.wav" "/content/PedalNetRT/data/s chain3.wav"


In [ ]:
#@title 4. **train model** Beware, checkpoint files not visible via Colab webui (-rw-r--r--) 


os.chdir('/content/PedalNetRT')  
import pytorch_lightning
!python3 "train.py" --batch_size=32 --max_epochs=100 --learning_rate=3e-3 --num_channels=5 --kernel_size=4 --dilation_depth=8
#!python3 "train.py" --batch_size=32 --max_epochs=100 --learning_rate=3e-3 --num_channels=5 






In [ ]:
!ls -d /content/PedalNetRT/lightning_logs/version_1/checkpoints/*

#!chmod -R 755 lightning_logs 


'/content/PedalNetRT/lightning_logs/version_1/checkpoints/epoch=99.ckpt'
'/content/PedalNetRT/lightning_logs/version_1/checkpoints/epoch=99.tmp_end.ckpt'


In [ ]:
#@title 5. convert torch model (ckpt) to plugin model (json) 
# The .ckpt model must be converted to a .json model to run in the plugin. Usage:

#!python3 convert_pedalnet_to_wavnetva.py --model=your_trained_model.ckpt
!python3 convert_pedalnet_to_wavenetva.py --model=/content/PedalNetRT/lightning_logs/version_1/checkpoints/epoch=99.ckpt





2020-10-15 00:21:38.871550: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [ ]:
!python test.py --model=/content/PedalNetRT/lightning_logs/version_1/checkpoints/epoch=99.ckpt

2020-10-15 00:21:56.102070: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [ ]:
!python plot_wav.py

Error to signal (with pre-emphasis filter):  0.86500657
Error to signal (no pre-emphasis filter):  0.8650066
Creating spectrogram data..
plot_wav.py:116: RuntimeWarning: divide by zero encountered in log10
  plt.pcolormesh(times, frequencies, 10*np.log10(spectrogram))
<Figure size 1300x800 with 3 Axes>
<Figure size 1200x800 with 2 Axes>


In [ ]:
#@title 6. convert model from TPU to CPU format
import torch_xla
import torch_xla.core.xla_model as xm

from model import PedalNet
# Change path below to match model file
model = PedalNet.load_from_checkpoint('lightning_logs/version_0/checkpoints/epoch=1.ckpt')

# xm.save(model.state_dict(), 'tpu_to_cpu.ckpt') 
xm.save(model, 'tpu_to_cpu_model.ckpt') 
# issues loading model, see pytorch_lightning issues #2303 and #3044 (Might be completed by now)

# TODO: Add code to be able to load saved model sucessfully
